In [4]:
import cvxpy as cp
import numpy as np
from collections import Counter

Solve
$$
\begin{align*}
\min_{x}& &f(x) \\
\text{subject to}& &c(x) = 0 \\
& &M(x) \succeq 0 
\end{align*}
$$

In [65]:
# Notation: small font is a (row) vector, capital font is a (symmetric) matrix

class LinConstraint:
    # a @ x + b = 0
    def __init__(self, a, b):
        self._a = a
        self._b = b
    def __call__(self, x):
        return self._a.T @ x + self._b
    def grad(self, x):
        return self._a
    def hess(self, x):
        return np.zeros((self._a.shape[0], self._a.shape[0]))

class QuadConstraint:
    # x.T @ A @ x + b @ x + c = 0
    def __init__(self, A, b, c):
        self._A = A
        self._b = b
        self._c = c
    def __call__(self, x):
        return x.T @ self._A @ x + self._b.T @ x + self._c
    def grad(self, x):
        return self._b + 2 * self._A @ x
    def hess(self, x):
        return 2 * self._A

# And what about the M(x) >> 0 constraint?

In [ ]:
class ByrdOmojokunTRSDPSolver:
    """
    This class solves the optimization problem:

    min_x           g.T @ x     (where g is a constant vector)
    subject to      c(x) = 0    (where c_i(x) is at most quadratic)
                    M(x) >> 0   (where >> means positive semidefinite) (not implemented yet)
    """
    def __init__(self, eq_cons, g, num_variables):
        self._tr_radius = 20. # in (0, inf) -- initial trust region radius
        self._alpha = 2. # in (1, inf) -- controls increase in trust region radius after a good step
        self._beta = 0.8 # in (0, 1) -- controls decreased trust region radius in which to relax
        self._gamma = 0.8 # in (0, 1) -- controls decrease in trust region radius after a failed step
        self._eta = 0.8 # in (0, 1) -- controls acceptance threshold for a step
        self._rho = 0.3 # in (0, 1) -- used in penalty parameter "mu" update
        self._eps = 1e-3 # maximum 2-norm of constraint violation allowed
        self._eq_cons = eq_cons
        self._A = lambda x: np.vstack([c.grad(x).T for c in eq_cons])
        self._c = lambda x: np.vstack([c(x) for c in eq_cons])
        self._f = lambda x: g.T @ x
        self._fgrad = lambda x: g
        self._num_variables = num_variables
        self._maxiters = 100
    
    def _find_v(self, A, c):
        v = cp.Variable(self._num_variables)
        obj = cp.Minimize(cp.sum_squares(A @ v + c))
        # cons = [cp.norm2(v) <= self._beta * self._tr_radius, self._M(v) >> 0]
        cons = [cp.norm2(v) <= self._beta * self._tr_radius]
        prob = cp.Problem(obj, cons)
        prob.solve()
        return v.value

    def _find_step(self, x, v, A, c):
        p = cp.Variable(self._num_variables)
        obj = cp.Minimize(cp.vdot(self._fgrad(x), p) + 0.5 )
        # cons = [self._A(x) @ (p - v) == 0, self._M(p - v) >> 0, cp.norm2(p) <= self._tr_radius]
        cons = [A @ (p - v) == 0, cp.norm2(p) <= self._tr_radius]
        prob = cp.Problem(obj, cons)
        prob.solve()
        return p.value

    def _merit_function(self, x, mu, c): #phi(x; mu)
        return self._f(x) + mu * np.linalg.norm(c, 2)

    def _merit_model(self, x, p, mu, A, c, hessL): # q_mu
        return self._f(x) + self._fgrad(x).T @ p + 0.5 * p.T @ hessL @ p + mu * np.linalg.norm(A @ p + c, 2)

    def _adjust_penalty_parameter(self, mu, x, p, c, hessL):
        quad = p.T @ hessL @ p
        sigma = 0 if quad > -self._eps else 1
        return max(mu, (self._fgrad(x).T @ p + (sigma / 2) * quad)/((1 - self._rho) * np.linalg.norm(c, 2)))

    def _constraint_violation_2_norm(self, x):
        return np.linalg.norm(self._c(x), 2)

    def _accept_step(self, x, p, mu, A, c, hessL):
        ared = self._merit_function(x, mu, c) - self._merit_function(x + p, mu, c)
        pred = self._merit_model(x, np.zeros(self._num_variables), mu, A, c, hessL) - self._merit_model(x, p, mu, A, c, hessL)
        rho = ared / pred
        # if rho > self._eta and min_eig(self._M(x + p)) >= -self._delta:
        if rho > self._eta:
            x += p
            self._tr_radius *= self._alpha
        else:
            self._tr_radius = self._gamma * np.linalg.norm(p, 2)

    def _compute_lambda(self, A, fgrad):
        return np.linalg.inv(A @ A.T) @ A @ fgrad
    
    def _compute_hessL(self, x, lamb):
        return -sum([l * c.hess(x) for l, c in zip(lamb, self._eq_cons)])

    def solve(self, init):
        """
        Solve the problem by taking steps.
        """
        x = init
        mu = 0.
        for _ in range(self._maxiters):
            # print(f'Step {_}:')
            fk, fgradk, Ak, ck = self._f(x), self._fgrad(x), self._A(x), self._c(x)
            lambdak = self._compute_lambda(Ak, fgradk)
            hessL = self._compute_hessL(x, lambdak)
            v = self._find_v(Ak, ck)
            print(lambdak)
            print(hessL)
            p = self._find_step(x, v, Ak, ck)
            mu = self._adjust_penalty_parameter(mu, x, p, ck, hessL)
            # print('v', v)
            # print('p', p)
            # print('mu', mu)
            self._accept_step(x, p, mu, Ak, ck, hessL)
            # print(x)
        print(f"2-norm of constraint violation: {self._constraint_violation_2_norm(x)}") 
        return x

In [100]:
# minimize f(x) = (x + y)
# subject to x^2 + y^2 = 1
# The answer is f(x) = -sqrt(2)
cons = QuadConstraint(np.array([[1., 0.],[0., 1.]]), np.zeros(2), -1.)
g = np.array([1., 1.])
solver = ByrdOmojokunTRSDPSolver([cons], g, 2)

xval = solver.solve(np.ones(2))
print(xval)
print(g @ xval)

[0.5]
[[-1. -0.]
 [-0. -1.]]
[0.66666401]
[[-1.33332801 -0.        ]
 [-0.         -1.33332801]]
[0.66666401]
[[-1.33332801 -0.        ]
 [-0.         -1.33332801]]
[0.66666401]
[[-1.33332801 -0.        ]
 [-0.         -1.33332801]]
[0.66666401]
[[-1.33332801 -0.        ]
 [-0.         -1.33332801]]
[0.66666401]
[[-1.33332801 -0.        ]
 [-0.         -1.33332801]]
[0.66666401]
[[-1.33332801 -0.        ]
 [-0.         -1.33332801]]
[0.66666401]
[[-1.33332801 -0.        ]
 [-0.         -1.33332801]]
[0.66666401]
[[-1.33332801 -0.        ]
 [-0.         -1.33332801]]
[0.66666401]
[[-1.33332801 -0.        ]
 [-0.         -1.33332801]]
[0.66666401]
[[-1.33332801 -0.        ]
 [-0.         -1.33332801]]
[0.66666401]
[[-1.33332801 -0.        ]
 [-0.         -1.33332801]]
[0.66666401]
[[-1.33332801 -0.        ]
 [-0.         -1.33332801]]
[0.66666401]
[[-1.33332801 -0.        ]
 [-0.         -1.33332801]]
[0.66666401]
[[-1.33332801 -0.        ]
 [-0.         -1.33332801]]
[0.66666401]
[[-1.3

In [81]:
# minimize f(x) = 2x + 6y
# subject to x + y = 0
#            x = -1
# The answer is [x, y] = [-1, 1], f(x) = 4
cons1 = LinConstraint(np.ones(2), 0.)
cons2 = LinConstraint(np.array([1., 0.]), 1.)
g = np.array([2., 6.])
solver = ByrdOmojokunTRSDPSolver([cons1, cons2], g, 2)

xval = solver.solve(np.zeros(2))
print(xval)
print(g @ xval)

/tmp/ipykernel_822/4245248644.py:52: RuntimeWarning: invalid value encountered in scalar divide
  return max(mu, (self._fgrad(x).T @ p + (sigma / 2) * quad)/((1 - self._rho) * np.linalg.norm(c, 2)))
/tmp/ipykernel_822/4245248644.py:60: RuntimeWarning: invalid value encountered in scalar divide
  rho = ared / pred


2-norm of constraint violation: 0.0
[-1.  1.]
4.0


<hr>

In [3]:
class Basis:
    def __init__(self, ops: list[str]):
        self._ops = ops
        self._mapping = {op: i for i, op in enumerate(ops)}
        self._sz = len(ops)
    
    def rank(self, word: str):
        v = self._sz ** (len(word)) - 1
        for i, op in enumerate(reversed(word)):
            v += self._mapping[op] * (self._sz ** i)
        return v

    def unrank(self, pos: int):
        len = 0
        while self._sz ** len - 1 <= pos:
            len += 1
        len -= 1
        pos -= self._sz ** len - 1
        word = [None] * len
        for i in range(len):
            word[i] = self._ops[pos % self._sz]
            pos //= self._sz
        return ''.join(word[::-1])

def commutator(word1: str, word2: str):
    expression = Counter()
    for i in range(len(word1)):
        for j in range(len(word2)):
            # I took the following line from Hartnoll/Xi Yin's code
            word = word1[:i] + word2[j+1:] + word2[:j] + word1[i+1:]
            if word1[i] == 'x' and word2[j] == 'p':
                expression[word] += 1j
            elif word1[i] == 'p' and word2[j] == 'x':
                expression[word] -= 1j
    return expression

In [ ]:
# We have B = {Br, Bc}, Br = {Re<A>}, Bc = {Im<A>}
# So <A> = Br[A] + i Bc[A]
# The real and complex parts of a constraint hold seperately
# (u + iv)(Br[A] + i Bc[A]) = (c + id)
# (u Br[A] - v Bc[A]) = c
# (u Bc[A] + v Br[A]) = d

def normalization(B):
    # Set <I> = 1.
    ar = np.zeros(2 * N)
    br = np.zeros(2 * N)
    ac = np.zeros(2 * N)
    bc = np.zeros(2 * N)
    ar[B[""]] = ac[B["i"]] = 1.
    br[0] = 1.
    return LinConstraint(ar, br), LinConstraint(ac, bc)

def schwinger_dyson(B, hamil: list, word1: str):
    # Calculate <[H,O]> = 0.
    # We get a sum of (u+iv)(Br[A] + i Bc[A]) = 0.
    ar = np.zeros(2 * N)
    br = np.zeros(2 * N)
    ac = np.zeros(2 * N)
    bc = np.zeros(2 * N)
    full_exp = Counter()
    for coeff, w in hamil:
        terms = commutator(w, word1)
        for word, c in terms.items():
            full_exp[word] += c * coeff
    for word, c in full_exp.items():
        u, v = np.real(c), np.imag(c)
        ar[B[word]] = u
        ar[B['i'+word]] = -v
        ac[B['i'+word]] = u
        ac[B[word]] = v
    return LinConstraint(ar, br), LinConstraint(ac, bc)

def reality(B, word: str):
    # Return <trO> = <trO\dag>*.
    # Br[O] + i Bc[O] = Br[O\dag] - i Bc[O\dag]
    wdag = word[::-1]
    ar = np.zeros(2 * N)
    br = np.zeros(2 * N)
    ac = np.zeros(2 * N)
    bc = np.zeros(2 * N)
    ar[B[word]] = 1.
    ar[B[wdag]] = -1.
    ac[B[word]] = ac[B[wdag]] = 1.
    return LinConstraint(ar, br), LinConstraint(ac, bc)
    

def symmetry_constraint(G: list, word: str):
    # Calculate <trGO> = 0.
    ar = np.zeros(2 * N)
    br = np.zeros(2 * N)
    ac = np.zeros(2 * N)
    bc = np.zeros(2 * N)
    for coeff, w in G:
        u, v = np.real(coeff), np.imag(coeff)
        ar[B[w+word]] = u
        ar[B['i'+w+word]] = -v
        ac[B['i'+w+word]] = u
        ac[B[w+word]] = v
    return LinConstraint(ar, br), LinConstraint(ac, bc)

def moment_matrix(basis, B, L):
    # Return moment matrix of operators.
    # https://physics.stackexchange.com/questions/130614/complex-semi-definite-programming
    # https://dl.acm.org/doi/10.1145/380752.380838
    n = 2 ** (L//2 + 1) - 1
    Mr = cp.bmat([[B[2 * basis.rank(basis.unrank(i)[::-1] + basis.unrank(j))] for j in range(n)] for i in range(n)])
    Mc = cp.bmat([[B[2 * basis.rank(basis.unrank(i)[::-1] + basis.unrank(j)) + 1] for j in range(n)] for i in range(n)])
    return cp.bmat([[Mr, -Mc], [Mc, Mr]])

def trace_cyclicity(basis, L, word: str):
    # Calculate <trAB> - <trBA>. Returns C in xT@C@x + (this too) <trBA> - <trAB> = 0.
    n = 2 ** (L + 1) - 1
    word1 = word
    word2 = word[1:] + word[0]
    row_indr, col_indr, datar = [], [], []
    row_indc, col_indc, datac = [], [], []
    for i in range(1, len(word)):
        w1ind = basis.rank(word[1:i])
        w2ind = basis.rank(word[i+1:])
        k = 0
        if word[0] == 'x' and word[i] == 'p':
            k = 0.5
        elif word[0] == 'p' and word[i] == 'x':
            k = -0.5
        row_indr.extend([2 * w1ind, 2 * w2ind + 1, 2 * w1ind + 1, 2 * w2ind])
        col_indr.extend([2 * w2ind + 1, 2 * w1ind, 2 * w2ind, 2 * w1ind + 1])
        datar.extend([-k, -k, -k, -k])
        row_indc.extend([2 * w1ind, 2 * w2ind, 2 * w1ind + 1, 2 * w2ind + 1])
        col_indc.extend([2 * w2ind, 2 * w1ind, 2 * w2ind + 1, 2 * w1ind + 1])
        datac.extend([k, k, -k, -k])
    Cr = csr_array((datar, (row_indr, col_indr)), (2 * n, 2 * n))
    Dr = np.zeros((2 * n, 1))
    Dr[2 * basis.rank(word2), 0] = 1
    Dr[2 * basis.rank(word1), 0] = -1
    Cc = csr_array((datac, (row_indc, col_indc)), (2 * n, 2 * n))
    Dc = np.zeros((2 * n, 1))
    Dc[2 * basis.rank(word2) + 1, 0] = 1
    Dc[2 * basis.rank(word1) + 1, 0] = -1
    return [QuadConstraint(Cr, Dr), QuadConstraint(Cc, Dc)]

In [ ]:
def add_linear_constraints(basis, L, hamil, G):
    row_ind, col_ind, data = [0], [0], [1] # This handles the normalization
    numc = 1
    n = 2 ** (L + 1) - 1
    def add_terms(terms):
        nonlocal numc
        new_row = 0
        cnt = 0
        for pos, coeff in terms:
            if pos >= 2 * n:
                return
        for pos, coeff in terms:
            cnt += 1
            new_row = 1
            col_ind.append(pos)
            data.append(coeff)
            row_ind.append(numc)
        numc += new_row
    def preprocess(terms):
        # we have a list terms of form (term, coeff) where term is a string
        # what we have to do separate out the real and complex parts
        # and return a list (i, coeff) where i is the position in the vector B
        realterm = []
        complexterm = []
        for term, coeff in terms:
            i = basis.rank(term)
            cr = np.real(coeff)
            cc = np.imag(coeff)
            realterm.extend([(2 * i, cr), (2 * i + 1, -cc)])
            complexterm.extend([(2 * i, cc), (2 * i + 1, cr)])
        return realterm, complexterm
    # for i in range(n):
    #     word = basis.unrank(i)
    #     realterm, complexterm = preprocess(schwinger_dyson(hamil, word))
    #     add_terms(realterm)
    #     add_terms(complexterm)
    #     realterm, complexterm = preprocess(symmetry_constraint(G, word))
    #     add_terms(realterm)
    #     add_terms(complexterm)
    #     realterm, complexterm = reality(basis, word)
    #     add_terms(realterm)
    #     add_terms(complexterm)
    # Px = b
    P = csr_array((data, (row_ind, col_ind)), (numc, 2 * n))
    b = np.zeros((numc, 1))
    b[0, 0] = 1
    return LinConstraints(P, b)

def add_quad_constraints(basis, L):
    quad_constraints = []
    n = 2 ** (L + 1) - 1
    for i in range(n):
        word = basis.unrank(i)
        if len(word) < 2:
            continue
        quad_constraints.extend(trace_cyclicity(basis, L, word))
    return quad_constraints